In [1]:
# Import necessary dependencies
import sys
import os
import zipfile

import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

# Get the current working directory
current_dir = os.getcwd()

# Project directory
project_dir = os.path.abspath(os.path.join(current_dir, '..', '..'))
sys.path.append(project_dir)

# Python file for custom functions
from src.utils import *

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Directory to your zipfile
ZIPFILE_PATH = os.path.join(project_dir, 'datasets', 'ICPSR_38925-V1.zip')

# Directories to the parquet files
DS0001 = os.path.join(project_dir, 'datasets', 'ICPSR_38925/batch_header_extract_ds0001.parquet')
DS0002 = os.path.join(project_dir, 'datasets', 'ICPSR_38925/admin_extract_ds0002.parquet')
DS0003 = os.path.join(project_dir, 'datasets', 'ICPSR_38925/incident_extract_ds0003.parquet')
DS0004 = os.path.join(project_dir, 'datasets', 'ICPSR_38925/victim_extract_ds0004.parquet')
DS0005 = os.path.join(project_dir, 'datasets', 'ICPSR_38925/arrestee_extract_ds0005.parquet')
DS0006 = os.path.join(project_dir, 'datasets', 'ICPSR_38925/offender_extract_ds0006.parquet')

# Directory to your codebook
CODEBOOK_PATH = os.path.join(project_dir, '38925-Documentation-PI_Codebook.xlsx')  # extracted
# CODEBOOK_PATH = 'ICPSR_38925-V1/ICPSR_38925/38925-Documentation-PI_Codebook.xlsx'  # in the zipfile

In [3]:
# Access to zip file
czip = zipfile.ZipFile(ZIPFILE_PATH)
czip.namelist()

['ICPSR_38925-V1/',
 'ICPSR_38925-V1/ICPSR_38925/',
 'ICPSR_38925-V1/ICPSR_38925/38925-Documentation-PI_Codebook.xlsx',
 'ICPSR_38925-V1/ICPSR_38925/DS0001/',
 'ICPSR_38925-V1/ICPSR_38925/DS0001/38925-0001-Data.tsv',
 'ICPSR_38925-V1/ICPSR_38925/DS0001/38925-0001-Codebook-ICPSR.pdf',
 'ICPSR_38925-V1/ICPSR_38925/DS0002/',
 'ICPSR_38925-V1/ICPSR_38925/DS0002/38925-0002-Codebook-ICPSR.pdf',
 'ICPSR_38925-V1/ICPSR_38925/DS0002/38925-0002-Data.tsv',
 'ICPSR_38925-V1/ICPSR_38925/DS0003/',
 'ICPSR_38925-V1/ICPSR_38925/DS0003/38925-0003-Data.tsv',
 'ICPSR_38925-V1/ICPSR_38925/DS0003/38925-0003-Documentation-readme_gzip.txt',
 'ICPSR_38925-V1/ICPSR_38925/DS0003/38925-0003-Codebook-ICPSR.pdf',
 'ICPSR_38925-V1/ICPSR_38925/38925-related_literature.txt',
 'ICPSR_38925-V1/ICPSR_38925/38925-manifest.txt',
 'ICPSR_38925-V1/ICPSR_38925/38925-descriptioncitation.html',
 'ICPSR_38925-V1/ICPSR_38925/TermsOfUse.html',
 'ICPSR_38925-V1/ICPSR_38925/DS0004/',
 'ICPSR_38925-V1/ICPSR_38925/DS0004/38925-0004-D

In [4]:
# Look up column names and data types
incident_schema = pl.read_parquet_schema(DS0003)
incident_schema

{'V40261': String,
 'ALLOFNS-ALL OFFENSE CODES FOR THE INCIDENT': String,
 'BH005-DATE ORI WAS ADDED': Datetime(time_unit='us', time_zone=None),
 'BH006-DATE ORI WENT NIBRS': Datetime(time_unit='us', time_zone=None),
 'BH007-CITY NAME': String,
 'BH008-STATE ABBREVIATION': String,
 'BH009-POPULATION GROUP': String,
 'BH010-COUNTRY DIVISION': String,
 'BH011-COUNTRY REGION': String,
 'BH012-AGENCY INDICATOR': String,
 'BH013-CORE CITY': String,
 'BH014-COVERED BY ORI': String,
 'BH015-FBI FIELD OFFICE': Int32,
 'BH016-JUDICIAL DISTRICT': String,
 'BH017-AGENCY NIBRS FLAG': String,
 'BH018-AGENCY INACTIVE DATE': Datetime(time_unit='us', time_zone=None),
 'BH019-CURRENT POPULATION 1': Int32,
 'BH020-UCR COUNTY CODE 1': Int32,
 'BH021-MSA CODE 1': Int32,
 'BH022-LAST POPULATION 1': Int32,
 'BH023-CURRENT POPULATION 2': Int32,
 'BH024-UCR COUNTY CODE 2': Int32,
 'BH025-MSA CODE 2': Int32,
 'BH026-LAST POPULATION 2': Int32,
 'BH027-CURRENT POPULATION 3': Int32,
 'BH028-UCR COUNTY CODE 3': In

In [5]:
# Columns with important data
important_data = [
    'STATE-NUMERIC STATE CODE', 'INCNUM-INCIDENT NUMBER', 'INCDATE-INCIDENT DATE',
    'BH007-CITY NAME', 'BH008-STATE ABBREVIATION', 'V1008-TOTAL OFFENSE SEGMENTS',
    'V1009-TOTAL VICTIM SEGMENTS', 'V1010-TOTAL OFFENDER SEGMENTS', 'V1011-TOTAL ARRESTEE SEGMENTS',
    'V20061-UCR OFFENSE CODE - 1', 'V20111-LOCATION TYPE - 1', 'V20141-TYPE CRIMINAL ACTIVITY/GANG INFO 1 - 1',
    'V20171-WEAPON / FORCE 1 - 1', 'V20201-BIAS MOTIVATION 1 - 1', 'V30121-SUSPECTED DRUG TYPE 1 - 1',
    'V40171-TYPE OF VICTIM-1', 'V40181-AGE OF VICTIM-1', 'V40191-SEX OF VICTIM-1',
    'V40201-RACE OF VICTIM-1', 'V40211-ETHNICITY OF VICTIM-1', 'V40221-RESIDENT STATUS OF VICTIM-1',
    'V40321-RELATIONSHIP VIC TO OFF 1-1', 'V50071-AGE OF OFFENDER - 1', 'V50081-SEX OF OFFENDER - 1',
    'V50091-RACE OF OFFENDER - 1', 'V50111-ETHNICITY OF OFFENDER - 1', 'V60081-ARREST DATE - 1',
    'V60121-ARRESTEE ARMED WITH 1 - 1', 'V60141-AGE OF ARRESTEE - 1', 'V60151-SEX OF ARRESTEE - 1',
    'V60161-RACE OF ARRESTEE - 1', 'V60171-ETHNICITY OF ARRESTEE - 1', 'V60181-RESIDENT STATUS OF ARRESTEE - 1',
    'V60191-DISPOSITION OF ARRESTEE UNDER 18 - 1'
]

In [8]:
# Read in city names
cities = pl.scan_parquet(
    DS0003,
).select(
    'BH007-CITY NAME'
).collect()

# Inspect unique names
unique_names = cities['BH007-CITY NAME'].str.to_lowercase().unique()
unique_names

BH007-CITY NAME
str
"""sugar hill"""
"""lake providence"""
"""fultondale"""
"""bison"""
"""roseland"""
…
"""saint francisville"""
"""oquawka"""
"""gordonsville"""


In [9]:
# Filter city names containing 'north', 'south', 'east', or 'west'
regions = ['north', 'south', 'east', 'west']
filtered_city_names = unique_names.filter(
    unique_names.str.contains('|'.join(regions))
)
filtered_city_names

BH007-CITY NAME
str
"""south fulton"""
"""westborough"""
"""south berwick"""
"""west berlin"""
"""west dover"""
…
"""key west"""
"""midwest city"""
"""northlake"""


In [10]:
# Filter city names with 'boston', 'chicago', and 'nashville'
targets = ['boston', 'chicago', 'nashville']
targeted_city_names = unique_names.filter(
    unique_names.str.contains('|'.join(targets))
)
targeted_city_names

BH007-CITY NAME
str
"""boston"""
"""west chicago"""
"""south boston"""
"""nashville"""
"""chicago"""
"""east chicago"""
"""chicago ridge"""
"""chicago heights"""
"""new boston"""


#### **Task:** Analyze the cities vs. all of their related names
- **New Boston** is in New Hampshire.
- **East Chicago** is in Indiana.
- There's only one occurrence of **Nashville**.